Datasets:
- CSV 1: international_matches.csv

Columnas: ID, Tournament, Date, Home Team, Home Goals, Away Goals, Away Team, Win Conditions, Home Stadium
- CSV 2: 2022_world_cup_matches.csv
Columnas: ID, Year, Date, Stage, Home Team, Away Team, Host Team

En este ejercicio se pide combinar estos datos con información adicional de los continentes a los que pertenecen los equipos y luego asignar una probabilidad de victoria basada en los históricos.


Objetivos:
1. Cargar datos y explorarlos.
2. Limpieza y preparación de datos
3. Preparación de datos adicionales y merge
4. Cálculo de probabilidades de victoria bada en histórico
5. Análisis exploratorio.
6. Visualización.


## 1. Carga de los datos


In [ ]:
import pandas as pd

# 1. Cargar el CSV de partidos internacionales
# Usamos parse_dates=['Date'] para que pandas interprete esa columna como fechas automáticamente
df_international = pd.read_csv('international_matches.csv', parse_dates=['Date'])

# 2. Cargar el CSV de partidos del mundial
df_world_cup = pd.read_csv('2022_world_cup_matches.csv', parse_dates=['Date'])

# 3. Revisar la estructura de 'df_international'
print("--- INFO: International Matches ---")
df_international.info()
print("\n--- HEAD: International Matches (Primeras 5 filas) ---")
print(df_international.head())

# 4. Revisar la estructura de 'df_world_cup'
print("\n" + "="*50 + "\n") # Separador visual
print("--- INFO: World Cup Matches ---")
df_world_cup.info()
print("\n--- HEAD: World Cup Matches (Primeras 5 filas) ---")
print(df_world_cup.head())

--- INFO: International Matches ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17769 entries, 0 to 17768
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   ID              17769 non-null  int64         
 1   Tournament      17769 non-null  object        
 2   Date            17769 non-null  datetime64[ns]
 3   Home Team       17769 non-null  object        
 4   Home Goals      17769 non-null  int64         
 5   Away Goals      17769 non-null  int64         
 6   Away Team       17769 non-null  object        
 7   Win Conditions  201 non-null    object        
 8   Home Stadium    17769 non-null  bool          
dtypes: bool(1), datetime64[ns](1), int64(3), object(4)
memory usage: 1.1+ MB

--- HEAD: International Matches (Primeras 5 filas) ---
   ID Tournament       Date Home Team  Home Goals  Away Goals Away Team  \
0   1   Friendly 1872-11-30  Scotland           0           0   England   


## 2. Limpieza y preparación de datos

In [ ]:
# Conteo de valores nulos por columna
print("Valores nulos por columna:")
print(df.isna().sum())

# Rellenar NaN en columnas numéricas clave con valores por defecto
df.fillna({'screen_time(min)': 0, 'notifications': 0}, inplace=True)

# Eliminar filas que no tengan app o tiempo de pantalla (esenciales para análisis)
df.dropna(subset=['app_name', 'screen_time(min)'], inplace=True)

print('\nDespués de limpieza de NaN:')
print(df.isna().sum())

display(df.head())


Valores nulos por columna:
user_id              0
app_name             0
screen_time(min)     2
notifications        2
date                 1
comentarios         17
dtype: int64

Después de limpieza de NaN:
user_id              0
app_name             0
screen_time(min)     0
notifications        0
date                 1
comentarios         17
dtype: int64


,user_id,app_name,screen_time(min),notifications,date,comentarios
0,1001,Instagram,45.0,10.0,2024-05-12,NaN
1,1002,WhatsApp,60.0,12.0,2024-05-12,NaN
2,1003,YouTube,120.0,8.0,2024-05-13,NaN
3,1004,Telegram,30.0,5.0,2024-05-14,NaN
4,1021,Instagram,0.0,14.0,2024-05-12,NaN


## 3. Preparación de datos adicionales y merge

In [8]:
# Detectar filas duplicadas completas
duplicados = df[df.duplicated()]
print("Filas duplicadas detectadas:")
display(duplicados)

# Eliminar duplicados
df.drop_duplicates(inplace=True)

print("Tamaño tras eliminar duplicados:", df.shape)


Filas duplicadas detectadas:


,user_id,app_name,screen_time(min),notifications,date,comentarios
7,1002,WhatsApp,60.0,12.0,2024-05-12,NaN


Tamaño tras eliminar duplicados: (16, 6)


## 4. Cálculo de probabilidades de victoria basada en histórico

In [ ]:
import pandas as pd

# ---------------------------------------------------------
# 1. CARGA DE DATOS
# ---------------------------------------------------------
df_international = pd.read_csv('international_matches.csv', parse_dates=['Date'])

# ---------------------------------------------------------
# 2. CÁLCULO DE ESTADÍSTICAS POR PAR DE EQUIPOS
# ---------------------------------------------------------
# 2.1 Crear columnas de resultados (1 o 0)
df_international['home_win'] = (df_international['Home Goals'] > df_international['Away Goals']).astype(int)
df_international['draw'] = (df_international['Home Goals'] == df_international['Away Goals']).astype(int)
df_international['away_win'] = (df_international['Home Goals'] < df_international['Away Goals']).astype(int)

# 2.2 Agrupar para obtener el historial
team_stats = df_international.groupby(['Home Team', 'Away Team']).agg(
    matches_count=('ID', 'count'),
    total_home_wins=('home_win', 'sum'),
    total_draws=('draw', 'sum'),
    total_away_wins=('away_win', 'sum')
).reset_index()

# 2.3 Calcular porcentaje y formatear
raw_prob = team_stats['total_home_wins'] / team_stats['matches_count']
team_stats['prob_local_win'] = (raw_prob * 100).round(2).astype(str) + '%'

# ---------------------------------------------------------
# 3. UNIÓN FINAL (ENRIQUECER LA TABLA PRINCIPAL)
# ---------------------------------------------------------
df_final = df_international.merge(
    team_stats, 
    on=['Home Team', 'Away Team'], 
    how='left'
)

# ---------------------------------------------------------
# 4. VISUALIZACIÓN EXACTA (COMO LA IMAGEN)
# ---------------------------------------------------------
# Definimos exactamente qué columnas queremos ver y en qué orden
columnas_a_mostrar = [
    'Date', 
    'Home Team', 
    'Away Team', 
    'matches_count', 
    'total_home_wins', 
    'total_draws', 
    'total_away_wins', 
    'prob_local_win'
]

print("--- TABLA GENERAL (Primeras 5 filas) ---")
print(df_final[columnas_a_mostrar].head())

print("\n--- CASO ESPECÍFICO: BRASIL VS ARGENTINA ---")
# Filtramos y mostramos CON LAS MISMAS COLUMNAS DE ESTADÍSTICA
filtro_bra_arg = df_final[(df_final['Home Team'] == 'Brazil') & (df_final['Away Team'] == 'Argentina')]
print(filtro_bra_arg[columnas_a_mostrar].head())

--- Dataframe Final (Primeras filas) ---
        Date Home Team Away Team  matches_count  total_home_wins  total_draws  \
0 1872-11-30  Scotland   England             58               24           14   
1 1873-03-08   England  Scotland             59               28           13   
2 1874-03-07  Scotland   England             58               24           14   
3 1875-03-06   England  Scotland             59               28           13   
4 1876-03-04  Scotland   England             58               24           14   

   total_away_wins prob_local_win  
0               20         41.38%  
1               18         47.46%  
2               20         41.38%  
3               18         47.46%  
4               20         41.38%  

--- Verificación: Filtrando partidos de Brasil vs Argentina ---
          Date Home Team  Away Team  matches_count  total_home_wins  \
384 1919-05-18    Brazil  Argentina             48               28   

     total_draws  total_away_wins prob_local_win

## 5. Análisis exploratorio

In [2]:
# Eliminamos columnas que estén completamente vacías
df.dropna(axis=1, how='all', inplace=True)

print("Columnas actuales del DataFrame:")
print(df.columns.tolist())


NameError: name 'df' is not defined

## 6. Visualización

In [ ]:
# Normalizar nombres de apps:
# - quitar espacios en blanco
# - poner la primera letra en mayúscula de cada palabra
# - eliminar espacios internos (Instagram  -> Instagram, 'YouTube Music' -> 'Youtubemusic')
df['app_name'] = (
    df['app_name']
      .astype(str)
      .str.strip()
      .str.title()
      .str.replace(' ', '', regex=False)
)

display(df[['app_name']].head(10))

# Buscar todas las filas que contienen "Youtube"
youtube_rows = df[df['app_name'].str.contains('Youtube', case=False, na=False)]
print("Filas con Youtube:")
display(youtube_rows)


In [ ]:
# Longitud del nombre de la app
df['app_name_len'] = df['app_name'].str.len()

# Apps que empiezan por 'Y'
apps_y = df[df['app_name'].str.startswith('Y', na=False)]

print("Apps que empiezan por 'Y':")
display(apps_y[['app_name', 'app_name_len']].drop_duplicates())

display(df[['app_name', 'app_name_len']].head())


In [ ]:
print("Tipos actuales:")
print(df.dtypes)

print("\nTipo real de cada celda en las primeras filas:")
display(df.applymap(type).head())

# Convertir automáticamente al tipo más adecuado (opcional)
df = df.convert_dtypes()

print("\nTipos tras convert_dtypes():")
print(df.dtypes)


In [ ]:
df.to_csv('screentime_analysis_clean.csv', index=False)
print('Archivo screentime_analysis_clean.csv guardado correctamente.')

In [3]:
# ¿Qué apps tienen más tiempo medio de pantalla?
screen_time_media = (
    df.groupby('app_name')['screen_time(min)']
      .mean()
      .sort_values(ascending=False)
)

print("Tiempo medio de pantalla (min) por app:")
display(screen_time_media)

# ¿Qué apps generan más notificaciones?
notif_media = (
    df.groupby('app_name')['notifications']
      .mean()
      .sort_values(ascending=False)
)

print("Notificaciones medias por app:")
display(notif_media)


Tiempo medio de pantalla (min) por app:


app_name
YouTube Music    130.0
YouTube          120.0
Netflix          110.0
Tik tok          100.0
YouTube Kids      95.0
  WhatsApp        80.0
WhatsApp          80.0
Telegram          60.0
Calendar Pro      50.0
Instagram         45.0
 instagram        30.0
TikTok             NaN
Name: screen_time(min), dtype: float64

Notificaciones medias por app:


app_name
  WhatsApp       15.0
Tik tok          12.0
Instagram        12.0
WhatsApp         12.0
YouTube Music     9.0
YouTube           8.0
YouTube Kids      7.0
Netflix           6.0
 instagram        6.0
Telegram          5.0
Calendar Pro      3.0
TikTok            NaN
Name: notifications, dtype: float64